In [ ]:
#Import functions to use for this analysis
import pandas as pd
import numpy as np
import time
import datetime as dt

In [ ]:
#Import csv
df_load = pd.read_csv('~/Documents/swimdata.csv')

In [ ]:
#Look at first rows of dataframe similar to R head() function 
df_load[0:6]

In [ ]:
#Look at last rows not inclusive of index
df_load[-4:-1]

In [ ]:
#Generate new dataframe with only relevant columns
df = df_load[['Place','Name','Category','Gender','Time']].iloc[0:275]

In [6]:
#Count time but also get median and standard deviation
participant_count = df.groupby(['Category']).count()['Time']

In [7]:
#Using logical operator on DNS, get percent of people by category that finished race
finish_percent = df[df.Time <> 'DNS'].groupby(['Category']).count()['Time']/df.groupby(['Category']).count()['Time']

In [8]:
cat_summary = pd.concat([finish_percent,participant_count],axis=1)
cat_summary.columns = ['Finish Rate','Participant Count'] #Add median and standard deviation of time
cat_summary

,Finish Rate,Participant Count
Category,,
F1112,1.000000,1
F1314,1.000000,14
F1517,1.000000,9
F1824,1.000000,15
F2529,0.923077,13
F3034,1.000000,11
F3539,1.000000,4
F4044,0.900000,10
F4549,0.857143,7


In [9]:
#Turn time from timestamp format into array form to get total seconds
df['TimeArray'] = df['Time'].str.split(':',1)

In [37]:
#Turn array into seconds
minutes_list = [item[0] for item in df['TimeArray']]
seconds_list = [item[-1] for item in df['TimeArray']]

df['minutelist'] = pd.Series(minutes_list, index=df.index)
df['secondslist'] = pd.Series(seconds_list, index=df.index)

#Bad option:
#np.where(df['Time'] <> 'DNS', 60*int(df['minutelist'])+int(df['secondslist']), 'DNS')

df['time_seconds'] = pd.to_numeric(df['minutelist'], errors='coerce')*60 + pd.to_numeric(df['secondslist'], errors='coerce')

In [39]:
#Look at median and variance of times
#time_detail = df[df.Time <> 'DNS'].groupby(['Category']).agg({'mean':np.median('time_seconds'),'SD':np.sd('time_seconds')})

,Place,Name,Category,Gender,Time,TimeArray,minutelist,secondslist,time_seconds
0,1,Jude Williams,M1517,Male,19:15.7,"[19, 15.7]",19,15.7,1155.7
1,2,Steve Allnutt,M2529,Male,19:42.6,"[19, 42.6]",19,42.6,1182.6
2,3,Sandoval Martha,F1824,Female,20:02.3,"[20, 02.3]",20,02.3,1202.3
3,4,Daniel Escalante,M1517,Male,20:29.3,"[20, 29.3]",20,29.3,1229.3
4,5,CLAUDIA MACIAS,F1517,Female,20:29.4,"[20, 29.4]",20,29.4,1229.4
5,6,EMILIO CAMARILLO,M1517,Male,20:33.9,"[20, 33.9]",20,33.9,1233.9
6,7,Bejarano Hernandez Andre,M1314,Male,20:34.5,"[20, 34.5]",20,34.5,1234.5
7,8,VERONICA MACIAS,F1517,Female,20:34.9,"[20, 34.9]",20,34.9,1234.9
8,9,Sandoval Aide,F1517,Female,20:35.4,"[20, 35.4]",20,35.4,1235.4
9,10,Alfred Hoejberg,M1517,Male,20:36.3,"[20, 36.3]",20,36.3,1236.3


In [ ]:
participant_locations = df_load['Team name'].iloc[0:275]

In [ ]:
#Group people by coming from California, Baja California (MX), and other US states
#for i in range(0,276):
#    if 'Baja California' in participant_locations[i]:    